In [352]:
import pandas as pd
import numpy as np
from statsmodels.stats.power import TTestIndPower
import matplotlib.pyplot as plt

#connect to snowflake - some kind of package dependency i need to fix - reach out to data engineering
#somehow pull model output right from the tool

#spit out camunda file - just need the routing % for rollout/holdout/test

In [353]:
# import snowflake.connector
# import sys
# con = snowflake.connector.connect(
#     user="timothy_peterson", #You can get it by executing in UI: desc user <username>;
#     account="missionlane.us-east-1", #Add all of the account-name between https:// and snowflakecomputing.com
#     authenticator="externalbrowser",
# )
# cur = con.cursor()
# print(sys.executable)
# print(sys.version)
# print(sys.version_info)
# try:
#     cur.execute("select current_date")
#     one_row=cur.fetchone()
#     print("Current_Date:",one_row[0])
#     cur.execute("SELECT current_version()")
#     one_row = cur.fetchone()
#     print("Snowfalke_Version:",one_row[0])
# finally:
#     cur.close()
# cur.close()

In [ ]:
# 3 month roa naco drift calculation


In [354]:
#inputs
year = 'Y1'
rec_mult = 1.0
pbad_stress = 1.0
rewards_rate = 0.0


# naco constraint
naco = 0.12

#rg clip amount constraint RG >= 5, no 2000 CLIP - can do int(mystring[-1]) to get the int
rg_clip_dict = {'risk5' : 2000, 'risk6' : 2000, 'risk7' : 2000, 'risk8' : 2000, 'risk9' : 2000}

#ROA/NACO contraints (lowest to highest)
s7_clips = [500, 1000]
s7_dict = {'3000 1000' : 0.9, '3000 500' : 0.5, '4000 1000' : 0.9, '4000 500' : 0.75}

s11_clips = [500, 1000, 2000]
s11_dict = {'3000 2000' : 0.9, '3000 1000' : 0.6, '3000 500' : 0.5, '4000 2000' : 0.9, '4000 1000' : 0.75
          , '4000 500' : 0.75}


In [355]:
#model output
#model_out = pd.read_csv('/Users/Timothy.Peterson/Downloads/default_output.csv')
#model_out = pd.read_csv('/Users/Timothy.Peterson/Downloads/COD_change_output.csv')
model_out = pd.read_csv('/Users/Timothy.Peterson/Downloads/pbad_adj_model_out.csv')

model_out.head()

,Unnamed: 0,clip_amount,Year,risk_group,pre clip line,Recession Multipliers,pbad stress all,util_group,Rewards Rate,Segment NPV,...,Average OS,AVG_OS_y1,AVG_OS_y2,AVG_OS_y3,Marginal_OS_y1,Marginal_OS_y2,Marginal_OS_y3,MU_y1,MU_y2,MU_y3
0,0,0,Y1,risk1,1000,1.0,1.0,0-10%,0.000,90.5847,...,145.6994,111.2882,156.0343,158.9730,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,Y1,risk1,1000,1.0,1.0,0-10%,0.010,47.6577,...,145.6994,111.2882,156.0343,158.9730,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,Y1,risk1,1000,1.0,1.0,0-10%,0.015,26.1942,...,145.6994,111.2882,156.0343,158.9730,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,Y1,risk1,1000,1.0,1.0,10-30%,0.000,146.4348,...,218.0694,213.0939,252.9944,235.1174,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,Y1,risk1,1000,1.0,1.0,10-30%,0.010,103.5079,...,218.0694,213.0939,252.9944,235.1174,0.0,0.0,0.0,0.0,0.0,0.0


In [356]:
#population output, use either a snowflake connection or use a downloaded csv file
s7_pop = pd.read_csv('/Users/Timothy.Peterson/Downloads/stmt_7_pop.csv')
s7_pop.head()

,RISK_GROUP,PCL_GROUP,UTIL_GROUP,COUNT(*)
0,risk1,1000,0-10%,718
1,risk1,1000,10-30%,108
2,risk1,1000,30-50%,63
3,risk1,1000,50-80%,48
4,risk1,1000,>80%,12


In [357]:
#population output, use either a snowflake connection or use a downloaded csv file
s11_pop = pd.read_csv('/Users/Timothy.Peterson/Downloads/stmt_11_pop.csv')
s11_pop.head()

,RISK_GROUP,PCL_GROUP,UTIL_GROUP,COUNT(*)
0,risk1,1000,0-10%,467
1,risk1,1000,10-30%,23
2,risk1,1000,30-50%,11
3,risk1,1000,50-80%,3
4,risk1,2000,0-10%,749


In [358]:
#iw constraint
iw = True
model_iw = model_out[(model_out['Year'] == year)&(model_out['Recession Multipliers'] == 1.5)&(model_out['pbad stress all'] == pbad_stress)&(model_out['Rewards Rate'] == rewards_rate)&(model_out['pre clip line'] <= 4000)&(model_out['util_group'] != '<=0%')]
model_iw['segment'] = model_iw['risk_group'].astype(str) + model_iw['pre clip line'].astype(str) + model_iw['util_group'].astype(str)


/var/folders/33/yz8_hgf50p10h3n647b6xhwm0000gq/T/ipykernel_10705/1101397929.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_iw['segment'] = model_iw['risk_group'].astype(str) + model_iw['pre clip line'].astype(str) + model_iw['util_group'].astype(str)


In [360]:

model_filt = model_out[(model_out['Year'] == year)&(model_out['Recession Multipliers'] == rec_mult)&(model_out['pbad stress all'] == pbad_stress)&(model_out['Rewards Rate'] == rewards_rate)&(model_out['pre clip line'] <= 4000)&(model_out['util_group'] != '<=0%')]

model_filt['segment'] = model_filt['risk_group'].astype(str) + model_filt['pre clip line'].astype(str) + model_filt['util_group'].astype(str)
model_filt['5_year_naco'] = model_filt['year_1_5_naco'] / (5*model_filt['Average OS'])
model_filt['5_year_roa_naco'] = model_filt['Year 1-5 NIAT'] / model_filt['year_1_5_naco']
segment_list = list(model_filt.segment.unique())


/var/folders/33/yz8_hgf50p10h3n647b6xhwm0000gq/T/ipykernel_10705/970294313.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_filt['segment'] = model_filt['risk_group'].astype(str) + model_filt['pre clip line'].astype(str) + model_filt['util_group'].astype(str)
/var/folders/33/yz8_hgf50p10h3n647b6xhwm0000gq/T/ipykernel_10705/970294313.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_filt['5_year_naco'] = model_filt['year_1_5_naco'] / (5*model_filt['Average OS'])
/var/folders/33/yz8_hgf50p10h

In [561]:
def clip_finder(stmt, model_output, iw_output, segments):
    out_list = []
    out_dict = {}
    #stmt 7 has different clip amounts than s11
    if stmt == 7:
        clip_list = sorted(s7_clips)
        for seg in segments:
            #add lowest to highest ordering
            count = 0
            for clip in clip_list:
                iw_seg = iw_output[(iw_output.segment == seg)&(iw_output.clip_amount == clip)]
                model_seg = model_output[(model_output.segment == seg)&(model_output.clip_amount == clip)]
                if model_seg['pre clip line'].unique()[0] <= 3000:
                    pcl = 3000
                elif model_seg['pre clip line'].unique()[0] == 4000:
                    pcl = 4000
                    
                roa_naco = s7_dict[str(pcl) + ' ' + str(clip)]

                #CLIP constraints
                if model_seg['5_year_naco'].unique()[0] < naco and model_seg['5_year_roa_naco'].unique()[0] > roa_naco and iw_seg['Year 1-5 NIAT'].unique()[0] > 0:
                    
                    #max clip amount in certain risk groups
                    if len(rg_clip_dict) > 0 and model_seg['risk_group'].unique()[0] in rg_clip_dict.keys():
                        
                        if clip < rg_clip_dict[model_seg['risk_group'].unique()[0]] :
                            out_dict[seg] = clip
                    
                            clip_dict = {'segment' : seg
                                ,'risk_group' : model_seg['risk_group'].unique()[0]
                                ,'pcl' : model_seg['pre clip line'].unique()[0]
                                ,'util_group' : model_seg['util_group'].unique()[0]
                                ,'clip_amount' : clip
                                ,'roa_naco' : model_seg['5_year_roa_naco'].unique()[0]
                                ,'iw_niat' : iw_seg['Year 1-5 NIAT'].unique()[0]
                                }
                    else:
                            out_dict[seg] = clip
                    
                            clip_dict = {'segment' : seg
                                ,'risk_group' : model_seg['risk_group'].unique()[0]
                                ,'pcl' : model_seg['pre clip line'].unique()[0]
                                ,'util_group' : model_seg['util_group'].unique()[0]
                                ,'clip_amount' : clip
                                ,'roa_naco' : model_seg['5_year_roa_naco'].unique()[0]
                                ,'iw_niat' : iw_seg['Year 1-5 NIAT'].unique()[0]
                                }
           
           #if you want to include minclips
            #if seg not in out_dict.keys():
            #    out_dict[seg] = 100
                    
            #    clip_dict = {'segment' : seg
            #                ,'risk_group' : model_seg['risk_group'].unique()[0]
            #                ,'pcl' : model_seg['pre clip line'].unique()[0]
            #                ,'util_group' : model_seg['util_group'].unique()[0]
            #                ,'clip_amount' : 100
            #                }
                        
           
            if clip_dict not in out_list:
                out_list.append(clip_dict)
                    

                
    if stmt == 11:
        clip_list = sorted(s11_clips)
        for seg in segments:
            for clip in clip_list:
                iw_seg = iw_output[(iw_output.segment == seg)&(iw_output.clip_amount == clip)]
                model_seg = model_output[(model_output.segment == seg)&(model_output.clip_amount == clip)]
                
                if model_seg['pre clip line'].unique()[0] <= 3000:
                    pcl = 3000
                elif model_seg['pre clip line'].unique()[0] == 4000:
                    pcl = 4000
                    
                roa_naco = s11_dict[str(pcl) + ' ' + str(clip)]
                
                if model_seg['5_year_naco'].unique()[0] < naco and model_seg['5_year_roa_naco'].unique()[0] > roa_naco and iw_seg['Year 1-5 NIAT'].unique()[0] > 0:
                        
                    if len(rg_clip_dict) > 0 and model_seg['risk_group'].unique()[0] in rg_clip_dict.keys():
                        
                        if clip < rg_clip_dict[model_seg['risk_group'].unique()[0]] :
                            out_dict[seg] = clip
                    
                            clip_dict = {'segment' : seg
                                ,'risk_group' : model_seg['risk_group'].unique()[0]
                                ,'pcl' : model_seg['pre clip line'].unique()[0]
                                ,'util_group' : model_seg['util_group'].unique()[0]
                                ,'clip_amount' : clip
                                ,'roa_naco' : model_seg['5_year_roa_naco'].unique()[0]
                                ,'iw_niat' : iw_seg['Year 1-5 NIAT'].unique()[0]
                                }
                    else:
                            out_dict[seg] = clip
                    
                            clip_dict = {'segment' : seg
                                ,'risk_group' : model_seg['risk_group'].unique()[0]
                                ,'pcl' : model_seg['pre clip line'].unique()[0]
                                ,'util_group' : model_seg['util_group'].unique()[0]
                                ,'clip_amount' : clip
                                ,'roa_naco' : model_seg['5_year_roa_naco'].unique()[0]
                                ,'iw_niat' : iw_seg['Year 1-5 NIAT'].unique()[0]
                                }
                    
                    
            if clip_dict not in out_list:
                out_list.append(clip_dict)
        

        
    out_df = pd.DataFrame(out_list)
                    
    return out_dict, out_df
                

In [562]:
output7, df7 = clip_finder(7, model_filt, model_iw, segment_list)
output11, df11 = clip_finder(11, model_filt, model_iw, segment_list)

In [584]:
#df7.head()
df7.to_csv('/Users/Timothy.Peterson/Downloads/stmt_7_clips_output.csv')
df11.to_csv('/Users/Timothy.Peterson/Downloads/stmt_11_clips.csv')

In [564]:
dfnew = pd.read_csv('/Users/Timothy.Peterson/Downloads/cod_adj_pol.csv')

In [565]:
mydict = {}
for seg in dfnew.segment.unique():
    newdf = dfnew[dfnew.segment == seg]
    mydict[seg] = newdf['clip_amount'].unique()[0]

In [566]:
sorted7 = df7.sort_values('roa_naco')
sorted7.head()

,segment,risk_group,pcl,util_group,clip_amount,roa_naco,iw_niat
66,risk5300050-80%,risk5,3000,50-80%,500,0.775081,374.8876
49,risk4100010-30%,risk4,1000,10-30%,500,0.775515,122.9263
59,risk4400030-50%,risk4,4000,30-50%,500,0.827778,463.7455
64,risk5200050-80%,risk5,2000,50-80%,500,0.838724,294.4697
46,risk3400030-50%,risk3,4000,30-50%,500,0.840149,444.5000


In [567]:
sorted11 = df11.sort_values('roa_naco')
sorted11.head()

,segment,risk_group,pcl,util_group,clip_amount,roa_naco,iw_niat
66,risk5300050-80%,risk5,3000,50-80%,1000,0.709822,392.9924
59,risk4400030-50%,risk4,4000,30-50%,1000,0.760571,470.4294
64,risk5200050-80%,risk5,2000,50-80%,1000,0.761354,325.4220
46,risk3400030-50%,risk3,4000,30-50%,1000,0.771826,451.2493
49,risk4100010-30%,risk4,1000,10-30%,500,0.775515,122.9263


In [568]:
#s7 grids
for pcl in s7_pop['PCL_GROUP'].unique():
    mydf = s7_pop[s7_pop['PCL_GROUP'] == pcl]
    newdf = pd.pivot_table(mydf, values='COUNT(*)', index='UTIL_GROUP', columns='RISK_GROUP', fill_value=0)
    print('\n' + str(pcl))
    print(newdf)


1000
RISK_GROUP  risk1  risk10  risk11  risk12  risk13  risk2  risk3  risk4  risk5  \
UTIL_GROUP                                                                      
0-10%         718      56      37      19      32   1352   1021    617    978   
10-30%        108      61      50      35      78    284    284    190    380   
30-50%         63      94      80      67     139    197    238    191    456   
50-80%         48     315     294     226     581    193    287    264    707   
>80%           12     751     707     584    2035     92    173    208    681   

RISK_GROUP  risk6  risk7  risk8  risk9  
UTIL_GROUP                              
0-10%         724    378    143     49  
10-30%        419    296    107     54  
30-50%        519    367    192     92  
50-80%       1031    918    546    268  
>80%         1355   1494   1050    515  

2000
RISK_GROUP  risk1  risk10  risk11  risk12  risk13  risk2  risk3  risk4  risk5  \
UTIL_GROUP                                          

In [569]:
#s7 grids
for pcl in df7.pcl.unique():
    mydf = df7[df7.pcl == pcl]
    newdf = pd.pivot_table(mydf, values='clip_amount', index='util_group', columns='risk_group', fill_value=0)
    print('\n' + str(pcl))
    print(newdf)


1000
risk_group  risk1  risk2  risk3  risk4  risk5
util_group                                   
0-10%        1000      0      0      0      0
10-30%       1000   1000    500    500      0
30-50%       1000   1000   1000   1000      0
50-80%       1000   1000   1000   1000   1000
>80%         1000   1000   1000   1000   1000

2000
risk_group  risk1  risk2  risk3  risk4  risk5
util_group                                   
0-10%        1000      0      0      0      0
10-30%       1000   1000      0      0      0
30-50%       1000   1000    500    500      0
50-80%       1000   1000   1000   1000    500
>80%         1000   1000   1000   1000   1000

3000
risk_group  risk1  risk2  risk3  risk4  risk5
util_group                                   
0-10%        1000      0      0      0      0
10-30%       1000   1000      0      0      0
30-50%       1000   1000    500    500      0
50-80%       1000   1000   1000   1000    500
>80%         1000   1000   1000   1000   1000

4000
risk_group

In [570]:
#s11 grids
for pcl in df11.pcl.unique():
    mydf = df11[df11.pcl == pcl]
    newdf = pd.pivot_table(mydf, values='clip_amount', index='util_group', columns='risk_group', fill_value=0)
    print('\n' + str(pcl))
    print(newdf)


1000
risk_group  risk1  risk2  risk3  risk4  risk5
util_group                                   
0-10%        2000      0      0      0      0
10-30%       2000   2000   1000    500      0
30-50%       2000   2000   2000   1000      0
50-80%       2000   2000   2000   2000   1000
>80%         2000   2000   2000   2000   1000

2000
risk_group  risk1  risk2  risk3  risk4  risk5
util_group                                   
0-10%        2000      0      0      0      0
10-30%       2000   2000      0      0      0
30-50%       2000   2000   1000   1000      0
50-80%       2000   2000   2000   2000   1000
>80%         2000   2000   2000   2000   1000

3000
risk_group  risk1  risk2  risk3  risk4  risk5
util_group                                   
0-10%        2000      0      0      0      0
10-30%       2000   2000      0      0      0
30-50%       2000   2000   1000   1000      0
50-80%       2000   2000   2000   2000   1000
>80%         2000   2000   2000   2000   1000

4000
risk_group

In [571]:
#for seg in segment_list:
#    if seg in output7.keys():
#        print('segment: ' + seg + ' CLIP Amount: ' + str(output7[seg]))
#    else:
#        print('segment: ' + seg + ' CLIP Amount: 0')

In [572]:
#for seg in segment_list:
#    if seg in output11.keys():
#        print('segment: ' + seg + ' CLIP Amount: ' + str(output11[seg]))
#    else:
#        print('segment: ' + seg + ' CLIP Amount: 0')

In [573]:
def policy_summary(pop, clip_dict, model_output):
    pop['segment'] = pop['RISK_GROUP'] + pop['PCL_GROUP'].astype(str) + pop['UTIL_GROUP'].astype(str)
    
    #count and exposure
    count = 0
    exposure = 0
    npv = 0
    naco = 0
    os = 0
    niat = 0
    outputlist = []
    
    for seg in clip_dict.keys():
        new_pop = pop[pop.segment == seg]
        old_count = new_pop['COUNT(*)'].unique()
        
        new_out = model_output[(model_output.segment == seg)&(model_output.clip_amount == clip_dict[seg])]
        
        #print(len(new_count))
        
        if len(old_count) > 0:
            new_count = old_count[0]
        else:
            new_count = 0
            
        count += new_count
        exposure += clip_dict[seg] * new_count
        npv += new_out['Marginal NPV'].unique()[0] * new_count
        naco += new_out['year_1_5_naco'].unique()[0] * new_count
        os += new_out['Average OS'].unique()[0] * new_count
        niat += new_out['Year 1-5 NIAT'].unique()[0] * new_count
        
    output = {'Count': count
             ,'Exposure': exposure
             ,'NPV': int(npv)
             ,'Avg OS': int(os)
             ,'5 Year NACO %': float(naco / (5*os))
             ,'5 Year NIAT %': float(niat / (5*os))
             ,'5 Year ROA/NACO %': float(niat / naco)
             ,'Avg CLIP': float(exposure / count)
             ,'Avg NPV': int(npv / count)}
    
    return output

In [574]:
summary7 = policy_summary(s7_pop, output7, model_filt)
summary11 = policy_summary(s11_pop, output11, model_filt)

In [575]:
print(summary7)

{'Count': 9484, 'Exposure': 8931000, 'NPV': 1431091, 'Avg OS': 7974332, '5 Year NACO %': 0.0842573123078148, '5 Year NIAT %': 0.1333826688931057, '5 Year ROA/NACO %': 1.5830396821325448, 'Avg CLIP': 941.6912695065373, 'Avg NPV': 150}


In [576]:
print(summary11)

{'Count': 7389, 'Exposure': 12549000, 'NPV': 1512545, 'Avg OS': 8211956, '5 Year NACO %': 0.0899987822430253, '5 Year NIAT %': 0.12328427253895986, '5 Year ROA/NACO %': 1.369843785286485, 'Avg CLIP': 1698.3353633779943, 'Avg NPV': 204}


In [577]:
summary7 = policy_summary(s7_pop, mydict, model_filt)

In [578]:
print(summary7)

{'Count': 11354, 'Exposure': 8893500, 'NPV': 1316178, 'Avg OS': 7980789, '5 Year NACO %': 0.08585261373140726, '5 Year NIAT %': 0.1332633237280552, '5 Year ROA/NACO %': 1.5522337403142308, 'Avg CLIP': 783.292231812577, 'Avg NPV': 115}


In [579]:
###Below is for camunda creation, which we do not need anymore

In [580]:
#example routing %s

rout_list = []
HO = 0.02
T = 0.48

for seg in segment_list:
    #print(seg)
    if int(seg[6]) > 0:
        rg = 2
    elif int(seg[6]) == 0:
        rg = 1
    else:
        rg = 0
    
    if rg == 1:
        risk = seg[:5]
        pcl = seg[5:9]
        util = seg[9:]
        
        rout_list.append({'risk_group' : risk
                     , 'pcl_group' : pcl
                     , 'util_group' : util
                     , 'segment' : seg
                     ,'HO' : HO
                     , 'RO' : 1- HO - T
                     , 'T' : T
                        })
    if rg == 2:
        risk = seg[:6]
        pcl = seg[6:10]
        util = seg[10:]
        
        rout_list.append({'risk_group' : risk
                     , 'pcl_group' : pcl
                     , 'util_group' : util
                     , 'segment' : seg
                     ,'HO' : HO
                     , 'RO' : 1- HO - T
                     , 'T' : T
                        })

In [581]:
rout_df = pd.DataFrame(rout_list)
rout_df

,risk_group,pcl_group,util_group,segment,HO,RO,T
0,risk1,1000,0-10%,risk110000-10%,0.02,0.5,0.48
1,risk1,1000,10-30%,risk1100010-30%,0.02,0.5,0.48
2,risk1,1000,30-50%,risk1100030-50%,0.02,0.5,0.48
3,risk1,1000,50-80%,risk1100050-80%,0.02,0.5,0.48
4,risk1,1000,>80%,risk11000>80%,0.02,0.5,0.48
...,...,...,...,...,...,...,...
255,risk9,4000,0-10%,risk940000-10%,0.02,0.5,0.48
256,risk9,4000,10-30%,risk9400010-30%,0.02,0.5,0.48
257,risk9,4000,30-50%,risk9400030-50%,0.02,0.5,0.48
258,risk9,4000,50-80%,risk9400050-80%,0.02,0.5,0.48


In [582]:
def camunda_creation(policy, routing):
    
    out_list = []
    #HO_list = []
    #RO_list = []
    #t_list = []
    
    out_list.append({
            'creditLineBucket' : [1..8]
            ,'riskGroup' : [1..13]
            ,'averageUtilization' : '<=0'
            ,'abTestingRandomNumber' : ''
            ,'averagePurchaseUtilization' : ''
            ,'testGroup' : 'ROLLOUT'
            ,'lineIncrease' : 100 #if we are using minCLIPs
            ,'transactorClip' : ''
            
        })
        
        
    for seg in routing.segment.unique():
        pol = policy[policy.segment == seg]
        mydf = routing[routing.segment == seg]
        
        
        pcl = mydf.pcl_group.unique()[0]
        risk = mydf.risk_group.unique()[0]
        util = mydf.util_group.unique()[0]
        
        out_list.append({
            'creditLineBucket' : pcl[0]
            ,'riskGroup' : risk[-1]
            ,'averageUtilization' : util
            ,'abTestingRandomNumber' :
            ,'averagePurchaseUtilization' :
            ,'testGroup' :
            ,'lineIncrease' :
            ,'transactorClip' :
            
        })
        
        
        

SyntaxError: invalid syntax (1813097870.py, line 9)

In [ ]:
creditLineBucket	riskGroup	averageUtilization	abTestingRandomNumber	averagePurchaseUtilization	testGroup	lineIncrease	transactorClip

In [ ]:
camunda_creation(df7, rout_df)